In [1]:
from gevent import monkey
monkey.patch_all(thread=False)

from gevent.pool import Pool
pool = Pool(1000)

from gevent.lock import BoundedSemaphore
semaphore_correct, semaphore_error, semaphore_remain = BoundedSemaphore(1), BoundedSemaphore(1), BoundedSemaphore(1)

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

directory_out = './file/3.make'

if os.path.exists(directory_out):
    shutil.rmtree(directory_out)
os.makedirs(directory_out)

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests
import json

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header

import time

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

# = = = = = = = = = = = = = = =

with open('./api.txt', 'r') as api_txt:
    api = api_txt.read()

# = = = = = = = = = = = = = = =

directory_in = './file/2.part'

list_file = [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_in}/{file}')]
for file in list_file:

    # = = = = = = = = = = = = = = =

    list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(f'{directory_in}/{file}',
                                                                           header=0,
                                                                           dtype=str).fillna('')

    df_input['No'] = df_input['No'].astype(int)
    df_input['Part Code'] = df_input['Part Code'].astype(int)

    count_remain = len(df_input)

    print(f'总数量：{count_remain}')
    print()

    # = = = = = = = = = = = = = = =

    def crawler(crawler_tuple):
        global api, list_dict_correct, list_series_error, count_remain

        crawler_status, (crawler_index, crawler_series) = False, crawler_tuple

        # = = = = = = = = = = = = = = =

        try:
            data = {'getArticleLinkedAllLinkingTargetManufacturer2': {'articleCountry': 'DE',
                                                                      'articleId': crawler_series['Part Code'],
                                                                      'country': 'DE',
                                                                      'countryGroupFlag': False,
                                                                      'linkingTargetType': 'VLOB',
                                                                      'provider': 23365}}

            count_retry = 0
            while True:
                count_retry += 1

                try:
                    url_check = crawler_series['Url']

                    resp = requests.post('https://webservice.tecalliance.services/pegasus-3-0/services/TecdocToCatDLB.jsonEndpoint',
                                         data=json.dumps(data),
                                         headers=get_header(**{'x-api-key': api}),
                                         timeout=(5, 15))

                    if resp.status_code == 200:
                        dict_ = resp.json()

                        if dict_['status'] == 401 and dict_['statusText'].strip() == 'Access not allowed':
                            print(f'fuck api {api}')
                            time.sleep(3)
                            with open('./api.txt', 'r') as api_txt:
                                api = api_txt.read()
                        elif dict_['status'] == 200:
                            break
                except KeyboardInterrupt:
                    break
                except:
                    continue

            # = = = = = = = = = = = = = = =

            if type(dict_['data']) == str and dict_['data'].strip() == '':
                list_make = []
            elif type(dict_['data']) == dict:
                list_make = dict_['data']['array']
            else:
                raise

            # = = = = = = = = = = = = = = =

            with semaphore_correct:
                list_dict_correct.extend([{'Part Code': crawler_series['Part Code'],
                                           'Make': make['manuName'].strip(),
                                           'Make Code': make['manuId'],
                                           'JOIN_PCMC': f'''{crawler_series['Part Code']};{make['manuId']}''',
                                           'Url': crawler_series['Url']} for make in list_make])

            # = = = = = = = = = = = = = = =

            crawler_status = True
        except:
            crawler_series['Check_Url'] = url_check

            with semaphore_error:
                list_series_error.append(crawler_series)

        # = = = = = = = = = = = = = = =

        if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and count_remain == 1 or count_remain % 300 == 0 and count_remain != 0:
            clear_output()
        print(f'''[{crawler_status}] - [请求{count_retry}次] - [剩余{count_remain - 1}条] - [{file.removesuffix('.xlsx')}（{(list_file.index(file) + 1) / len(list_file) * 100:.2f}%）] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {crawler_series['No']}. {url_check}\n''')

        with semaphore_remain:
            count_remain -= 1

    # = = = = = = = = = = = = = = =

    pool.map(crawler, df_input.iterrows())

    print('输出ing...')
    print()
    if list_dict_correct:
        df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['Part Code', 'Make'],
                                                                 ascending=[True, True],
                                                                 ignore_index=True)
        df_correct.to_excel(f'''{directory_out}/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if list_series_error:
        df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                               ascending=[True],
                                                               ignore_index=True)
        df_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-make_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = [file for file in sorted(os.listdir(directory_out)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_out}/{file}')]
for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'{directory_out}/{file}',
              f'''{directory_out}/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：10

fuck api 2BeBXg6KgXfwZr8i2P2L7H5W6ujr4XXcy3hTGWmJADw69bQB1Z2J
fuck api 2BeBXg6KgXfwZr8i2P2L7H5W6ujr4XXcy3hTGWmJADw69bQB1Z2J
fuck api 2BeBXg6KgXfwZr8i2P2L7H5W6ujr4XXcy3hTGWmJADw69bQB1Z2J
fuck api 2BeBXg6KgXfwZr8i2P2L7H5W6ujr4XXcy3hTGWmJADw69bQB1Z2J
fuck api 2BeBXg6KgXfwZr8i2P2L7H5W6ujr4XXcy3hTGWmJADw69bQB1Z2J
fuck api 2BeBXg6KgXfwZr8i2P2L7H5W6ujr4XXcy3hTGWmJADw69bQB1Z2J
fuck api 2BeBXg6KgXfwZr8i2P2L7H5W6ujr4XXcy3hTGWmJADw69bQB1Z2J
fuck api 2BeBXg6KgXfwZr8i2P2L7H5W6ujr4XXcy3hTGWmJADw69bQB1Z2J
fuck api 2BeBXg6KgXfwZr8i2P2L7H5W6ujr4XXcy3hTGWmJADw69bQB1Z2J
fuck api 2BeBXg6KgXfwZr8i2P2L7H5W6ujr4XXcy3hTGWmJADw69bQB1Z2J
fuck api 2BeBXg6KgXfwZr8i2P2L7H5W6ujr4XXcy3hTGWmJADw69bQB1Z2J
fuck api 2BeBXg6KgXfwZr8i2P2L7H5W6ujr4XXcy3hTGWmJADw69bQB1Z2J
fuck api 2BeBXg6KgXfwZr8i2P2L7H5W6ujr4XXcy3hTGWmJADw69bQB1Z2J
fuck api 2BeBXg6KgXfwZr8i2P2L7H5W6ujr4XXcy3hTGWmJADw69bQB1Z2J
fuck api 2BeBXg6KgXfwZr8i2P2L7H5W6ujr4XXcy3hTGWmJADw69bQB1Z2J
fuck api 2BeBXg6KgXfwZr8i2P2L7H5W6ujr4XXcy3hTGWmJADw69bQB1Z2J


Progress: 100%|██████████████████████████████| 2/2 [00:00<00:00, 3278.08it/s]

Done ~
